#### Character RNN 

In [2]:
import time 
import random 
import unidecode 
import string 
import re 
import matplotlib.pyplot as plt 
import torch 

torch.backends.cudnn.deterministic = True 

In [14]:
RANDOM_SEED= 234 
torch.manual_seed(RANDOM_SEED)
DEVICE= torch.device('cpu')

NUM_ITER= 5000  # epoch size 
LEARNING_RATE = 0.005 
EMBEDDING_DIM= 100  # size of the embedding weight 
HIDDEN_DIM= 100     # size of the hidden layer 
NUM_HIDDEN_LAYER= 1     # number of hidden layer after embedding 

TEXT_PORTION_SIZE= 200  # size of each text 


In [4]:
string.printable    # all the ascii characters this string class can take 

'0123456789abcdefghijklmnopqrstuvwxyzABCDEFGHIJKLMNOPQRSTUVWXYZ!"#$%&\'()*+,-./:;<=>?@[\\]^_`{|}~ \t\n\r\x0b\x0c'

In [11]:
with open('covid19-faq.txt', 'r') as f:     # open the file 
    textfile= f.read()  # read the file and load into a variable called textfile 

textfile = unidecode.unidecode(textfile)    # convert all the special characters 

# get rid of all the white spaces 
textfile = re.sub(" +"," ", textfile)

TEXT_LENGTH= len(textfile)  # find the total length of the textfile= text_length

TEXT_LENGTH

84658

In [15]:
random.seed(RANDOM_SEED)    # set the random seed 
# this will lead to weird texts 
def random_portion (textfile):  # sample text randomly 
    start_index= random.randint(0, TEXT_LENGTH-TEXT_PORTION_SIZE)   # randomly sample sentence with TEXT_PORTION_SIZE # of characters in it 
    end_index= start_index + TEXT_PORTION_SIZE + 1 
    return textfile[start_index:end_index]  # return the string of that size 

print(random_portion(textfile))

 to be submitted. In addition, all current campus health protocols must be followed including wearing facial covering. Additional information on the campus travel policy is located here.

Under current


In [16]:
# convert text into torch tensor for training purposes 
def char_to_tensor(text):
    lst= [string.printable.index(c) for c in text]  # convert the text into ASCII characters(ASCII indices) 
    tensor= torch.tensor(lst).long()   # convert this into a tensor
    return tensor 
print(char_to_tensor("abcDEF")) 

tensor([10, 11, 12, 39, 40, 41])


In [17]:
def draw_random_sample(textfile):
    text_long = char_to_tensor(random_portion(textfile))    # randomly get text and convert into torch tensor 
    input= text_long[:-1]   # input is every letter besides last one 
    targets= text_long[1:]   # output starts from index 1, we input one and predict the next one (that's why we cut off the last letter of )
    return input, targets 

In [18]:
draw_random_sample(textfile)

(tensor([32, 24, 94, 13, 10, 34, 28, 94, 28, 18, 23, 12, 14, 94, 29, 17, 14, 18,
         27, 94, 21, 10, 28, 29, 94, 14, 25, 18, 28, 24, 13, 14, 94, 24, 15, 94,
         31, 24, 22, 18, 29, 18, 23, 16, 94, 24, 27, 94, 13, 18, 10, 27, 27, 17,
         14, 10, 78, 94, 18, 15, 94, 29, 17, 14, 34, 94, 17, 10, 31, 14, 94, 11,
         14, 14, 23, 94, 24, 23, 94, 10, 23, 29, 18, 11, 18, 24, 29, 18, 12, 94,
         15, 24, 27, 94, 10, 29, 94, 21, 14, 10, 28, 29, 94,  2,  4, 94, 17, 24,
         30, 27, 28, 94, 18, 15, 94, 25, 27, 14, 28, 12, 27, 18, 11, 14, 13, 78,
         94, 24, 27, 94, 10, 28, 94, 10, 25, 25, 27, 24, 31, 14, 13, 94, 29, 24,
         94, 32, 24, 27, 20, 94, 11, 34, 94, 10, 94, 13, 24, 12, 29, 24, 27, 75,
         96, 96, 48, 24, 28, 29, 94, 14, 22, 25, 21, 24, 34, 14, 14, 28, 94, 32,
         18, 21, 21, 94, 11, 14, 94, 10, 11, 21, 14, 94, 29, 24, 94, 27, 14, 29,
         30, 27]),
 tensor([24, 94, 13, 10, 34, 28, 94, 28, 18, 23, 12, 14, 94, 29, 17, 14, 18, 27,
         

#### Model 

In [ ]:
class RNN(torch.nn.Module):
    def __init__ (self, input_size, embed_size, hidden_size, output_size, num_layers):
        super().__init__() 
        self.hidden_size= hidden_size
        self.num_layers= num_layers
        self.embed= torch.nn.Embedding(num_embeddings=input_size,embedding_dim=embed_size)